# Experiments with `fsspec`

In [2]:
import fsspec

In [3]:
# Setup S3 URLs and folder locations within the S3 bucket
S3_URL = "s3://drivendata-competition-biomassters-public-us"
train_features_s3 = S3_URL + "/train_features/"
train_agbm_s3 = S3_URL + "/train_agbm/"
test_features_s3 = S3_URL + "/test_features/"

In [17]:
kwargs = {'anon': True}
fs, path = fsspec.core.url_to_fs(S3_URL, **kwargs)

In [18]:
fs.isdir(path)

True

In [19]:
files = fs.ls(path)

In [21]:
files

['drivendata-competition-biomassters-public-us/features_metadata.csv',
 'drivendata-competition-biomassters-public-us/test_features',
 'drivendata-competition-biomassters-public-us/train_agbm',
 'drivendata-competition-biomassters-public-us/train_agbm_metadata.csv',
 'drivendata-competition-biomassters-public-us/train_features']

In [22]:
import time

In [23]:
s1 = time.time()

In [24]:
time.time() - s1

7.624447822570801